In [ ]:
#!/usr/bin/env python
 
'''
UTAS_DB_tools.py
A. J. McCulloch, February 2020
'''

####################################################################################################
# Import modules
####################################################################################################

import pandas as pd # Required for dataframe manipulation
import cx_Oracle # Required for connection to database
from getpass import getpass # Required for password input

####################################################################################################
# Define classes
####################################################################################################
class DW_connect:
    
    connected = False # Initialise the connection attribute flag
    
    # Establish connection to the data warehouse
    def __init__(self, username):
        if self.connected == False: # Don't connect if already connected
            self.username = username # Username to connect to data warehouse
            self.password = getpass() # Get password associated with user above

            """
            Make the required Data Source Name (DSN) Transparent Network Substrate (TNS)
            Info comes from an email from Nathalie (FW: DB Client Installs for Data Warehouse Access)
            Originating from Andrew with a document Oracle DB Drivers and Install
            """
            self.dsn_tns = cx_Oracle.makedsn('exa1-scan.its.utas.edu.au', '1521', service_name=r'edwprod_maa')
            # Connect to the database
            self.conn = cx_Oracle.connect(user = self.username, password = self.password, dsn = self.dsn_tns)

            # Verify connection has been made and update connection flag
            try:
                print("Connected to database, Oracle version {}".format(self.conn.version))
                self.connected = True
            except NameError:
                print("Could not connect to database")
        else:
            print("Connection already established")
    
    # Disconnect from the data warehouse
    def disconnect(self):
        self.conn.close()
        print("Connection to database ended")
        self.connected = False

# Class required to store data warehouse tables
class tablist(object):
    pass
####################################################################################################
# Define functions
####################################################################################################
# A function to run SQL query over a particular connection
def runSQL(query, conn = EDW.conn):
    # Use pandas to execute SQL
    return pd.read_sql_query(query, con = conn)
     
# Return currently accessible tables
def get_tables(schema='owner'):
    # SQL to retun all tables accessible under schema
    q = """
        SELECT
            table_name, {0}
        FROM
            all_tables
        ORDER BY
            {0}, table_name
        """.format(schema)
    # Run the SQL and return the result
    return runSQL(q)

# Function to return all available tables
def init_tables():
    tbls = get_tables() # Get available tables
    sch = tbls.OWNER.unique() # Get unique schema
    # Store each schema as an attribute for table object with associated table
    for s in sch:
        setattr(tables, s, tbls[tbls['OWNER']==s].reset_index(drop=True)) # Indexdrop required!
    print('Available schema are '+', '.join('{}'.format(s) for s in sch))

# Function to return an entire table
def get_all(table, row):
    q = 'SELECT * FROM {}.{}'.format(tables.STUDENTS.OWNER[row], tables.STUDENTS.TABLE_NAME[row])
    return runSQL(q)

# Connecting to the data warehouse
## Example code
### Initialise connection

In [ ]:
####################################################################################################
####################################################################################################
# Code starts here
####################################################################################################
####################################################################################################

# Connect to the data warehouse
EDW = DW_connect('ajm32')

### Find availables tables

In [ ]:
tables = tablist() # Create tablist object
init_tables()

### Return a table

In [ ]:
df = get_all(tables.STUDENTS, 0)
df.head(5)

### Disconnect from the database server

In [ ]:
# Disconnect from the data warehouse
EDW.disconnect()

# Manipulating the data